# Benchmark Validation — LLaMA 3.3 70B

Compare measured TTFT/ITL numbers against the analytical estimators for the 2×H100 FP8 benchmark in `tested_benchmarks`.

In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go

from utils.config import get_model_config, get_hardware_config
from utils.math_utils import (
    prefill_compute_time,
    prefill_memory_time,
    decode_compute_time,
    decode_memory_time,
)

pd.options.display.max_rows = 20
pd.options.display.max_columns = 20


## Load configuration and raw measurements

In [ ]:

DATA_PATH = Path("tested_benchmarks/nim_llama33_70b_v1.8.0_2xH100_fp8TP2.csv")
MODEL_NAME = "llama33_70B"
HARDWARE_LOOKUP = {
    "H100_80G": "H100_80GB_FP8_TP2",
}

df_raw = pd.read_csv(DATA_PATH)
df_raw["hardware_key"] = df_raw["hardware"].map(HARDWARE_LOOKUP)
if df_raw["hardware_key"].isna().any():
    missing = df_raw[df_raw["hardware_key"].isna()]["hardware"].unique()
    raise ValueError(f"Missing HARDWARE_LOOKUP entries for: {missing}")

model_cfg = get_model_config(MODEL_NAME)
hardware_cfg = get_hardware_config(df_raw["hardware_key"].iat[0])
display(df_raw.head())


## Compute analytical estimates

For each scenario we compute the prefill (TTFT) and decode (ITL) bottlenecks, keep both compute/memory components, and convert them to milliseconds.

In [ ]:

def estimate_row(row):
    S = float(row["concurrency"])
    L_prompt = float(row["input_tokens"])
    L_decode = L_prompt + float(row["output_tokens"])

    prefill_compute = prefill_compute_time(S, L_prompt, model_cfg, hardware_cfg)
    prefill_memory = prefill_memory_time(S, L_prompt, model_cfg, hardware_cfg)
    # decode_compute = decode_compute_time(S, L_decode, model_cfg, hardware_cfg)
    # decode_memory = decode_memory_time(S, L_decode, model_cfg, hardware_cfg)
    decode_compute = decode_compute_time(S, L_prompt, model_cfg, hardware_cfg)
    decode_memory = decode_memory_time(S, L_prompt, model_cfg, hardware_cfg, running_tokens_cap=3.03e5)

    ttft_compute_ms = float(prefill_compute * 1e3)
    ttft_memory_ms = float(prefill_memory * 1e3)
    itl_compute_ms = float(decode_compute * 1e3)
    itl_memory_ms = float(decode_memory * 1e3)

    return pd.Series({
        "prefill_compute_ms": ttft_compute_ms,
        "prefill_memory_ms": ttft_memory_ms,
        "ttft_model_ms": max(ttft_compute_ms, ttft_memory_ms),
        "decode_compute_ms": itl_compute_ms,
        "decode_memory_ms": itl_memory_ms,
        "itl_model_ms": max(itl_compute_ms, itl_memory_ms),
    })

df_eval = df_raw.join(df_raw.apply(estimate_row, axis=1))
df_eval["ttft_ratio"] = df_eval["ttft_model_ms"] / df_eval["TTFT_ms"]
df_eval["itl_ratio"] = df_eval["itl_model_ms"] / df_eval["ITL_ms"]
df_eval["ttft_delta_ms"] = df_eval["ttft_model_ms"] - df_eval["TTFT_ms"]
df_eval["itl_delta_ms"] = df_eval["itl_model_ms"] - df_eval["ITL_ms"]

# df_eval = df_eval.sort_values(["input_tokens", "concurrency"]).reset_index(drop=True)
display(df_eval[[
    "input_tokens", "concurrency", "TTFT_ms", "ttft_model_ms", "ttft_delta_ms",
    "ITL_ms", "itl_model_ms", "itl_delta_ms",
]])


## Error summary by prompt length

In [ ]:

summary = (
    df_eval
    .groupby("input_tokens")[[
        "TTFT_ms", "ttft_model_ms", "ttft_ratio",
        "ITL_ms", "itl_model_ms", "itl_ratio"
    ]]
    .agg(["mean", "median"])
)
summary


## Plots

The helpers below make it easy to compare measured vs. modelled values for any prompt length.

In [ ]:

def plot_metric_vs_concurrency(metric_name, observed_col, model_col, prompt_tokens):
    subset = df_eval[df_eval["input_tokens"] == prompt_tokens].sort_values("concurrency")
    if subset.empty:
        raise ValueError(f"No rows with input_tokens={prompt_tokens}")
    melted = subset.melt(
        id_vars=["concurrency"],
        value_vars=[observed_col, model_col],
        var_name="series",
        value_name="value",
    )

    title = f"{metric_name} vs concurrency (prompt={prompt_tokens})"
    fig = px.line(melted, x="concurrency", y="value", color="series", markers=True, title=title)
    fig.update_layout(xaxis_title="Concurrency", yaxis_title=f"{metric_name} [ms]")
    fig.show()

df_eval.to_csv("eval_results.csv")

for prompt_tokens in sorted(df_eval["input_tokens"].unique()):
    plot_metric_vs_concurrency("TTFT_ms", "TTFT_ms", "ttft_model_ms", prompt_tokens)

for prompt_tokens in sorted(df_eval["input_tokens"].unique()):
    plot_metric_vs_concurrency("ITL_ms", "ITL_ms", "itl_model_ms", prompt_tokens)


In [ ]:

fig_ttft = px.scatter(
    df_eval,
    x="TTFT_ms",
    y="ttft_model_ms",
    color="input_tokens",
    hover_data=["concurrency", "prefill_limit"],
    title="TTFT: measured vs modelled",
)
fig_ttft.add_trace(
    go.Scatter(
        x=[df_eval["TTFT_ms"].min(), df_eval["TTFT_ms"].max()],
        y=[df_eval["TTFT_ms"].min(), df_eval["TTFT_ms"].max()],
        mode="lines",
        name="ideal",
    )
)
fig_ttft.update_layout(xaxis_title="Measured TTFT [ms]", yaxis_title="Model TTFT [ms]")
fig_ttft.show()

fig_itl = px.scatter(
    df_eval,
    x="ITL_ms",
    y="itl_model_ms",
    color="input_tokens",
    hover_data=["concurrency", "decode_limit"],
    title="ITL: measured vs modelled",
)
fig_itl.add_trace(
    go.Scatter(
        x=[df_eval["ITL_ms"].min(), df_eval["ITL_ms"].max()],
        y=[df_eval["ITL_ms"].min(), df_eval["ITL_ms"].max()],
        mode="lines",
        name="ideal",
    )
)
fig_itl.update_layout(xaxis_title="Measured ITL [ms]", yaxis_title="Model ITL [ms]")
fig_itl.show()


---

Adjust `MODEL_NAME`, `HARDWARE_LOOKUP`, or the plotting helpers above to explore additional benchmark files.